# **DSpy Example**

Importing the required libraries:

In [1]:
import dspy

c:\Users\peraf\Vs Code\dspy-example\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading the model:

In [2]:
llama = dspy.LM(model='ollama/llama3.2',api_base_url='http://localhost:11434')
dspy.settings.configure(lm=llama)

## **Signature in DSpy**

In [3]:
document = r"""

Crítica hacia la Inacción de los Gobiernos en el Cambio Climático

  El cambio climático se ha convertido en una de las crisis más apremiantes de nuestro tiempo, pero los gobiernos parecen estar más preocupados por intereses económicos
  a corto plazo que por el bienestar del planeta. A pesar de las abrumadoras evidencias científicas, muchos líderes continúan posponiendo acciones decisivas.

  1. Promesas Vacías: A menudo, los gobiernos hacen promesas rimbombantes en cumbres internacionales, pero rara vez las cumplen. Estas palabras se convierten en una
  estrategia para desviar la atención del público, mientras siguen apoyando industrias contaminantes.

  2. Desinversión en Energías Renovables: Mientras el mundo clama por un cambio hacia energías sostenibles, muchos gobiernos siguen invirtiendo en combustibles fósiles,
  ignorando el potencial de las energías limpias y perjudicando el futuro de las próximas generaciones.

  3. Falta de Regulaciones Efectivas: Aunque algunos países han implementado regulaciones ambientales, la falta de rigor y supervisión significa que muchas industrias
  continúan contaminando sin repercusiones significativas. Los gobiernos deben asumir la responsabilidad y actuar con urgencia.

  4. Desigualdad Global: Los países en desarrollo, que a menudo son los más afectados por el cambio climático, son también los menos responsables de la crisis.
  Sin embargo, los gobiernos de naciones ricas parecen ignorar sus responsabilidades hacia estos países, perpetuando una injusticia que solo agrava la situación.

  La inacción de los gobiernos frente al cambio climático no solo es irresponsable, sino que también representa una traición a la confianza de sus ciudadanos
  y a la supervivencia del planeta. Es hora de que se tomen decisiones valientes y se actúe con seriedad.

"""

In [4]:
summarize = dspy.ChainOfThought("document -> summary")
response = summarize(document=document)

print(response)

Prediction(
    reasoning='La inacción de los gobiernos en el cambio climático es irresponsable y representa una traición a la confianza de sus ciudadanos y a la supervivencia del planeta. Los gobiernos deben asumir la responsabilidad y actuar con urgencia para abordar esta crisis.',
    summary='El cambio climático se ha convertido en una de las crisis más apremiantes de nuestro tiempo, pero los gobiernos parecen estar más preocupados por intereses económicos a corto plazo que por el bienestar del planeta. Es hora de que se tomen decisiones valientes y se actúe con seriedad para abordar esta crisis.'
)


In [5]:
llama.inspect_history(n=1)





System message:

Your input fields are:
1. `document` (str)

Your output fields are:
1. `reasoning` (str)
2. `summary` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## document ## ]]
{document}

[[ ## reasoning ## ]]
{reasoning}

[[ ## summary ## ]]
{summary}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `document`, produce the fields `summary`.


User message:

[[ ## document ## ]]


Crítica hacia la Inacción de los Gobiernos en el Cambio Climático

  El cambio climático se ha convertido en una de las crisis más apremiantes de nuestro tiempo, pero los gobiernos parecen estar más preocupados por intereses económicos
  a corto plazo que por el bienestar del planeta. A pesar de las abrumadoras evidencias científicas, muchos líderes continúan posponiendo acciones decisivas.

  1. Promesas Vacías: A menudo, los gobiernos hacen promesas rimbombantes en cumbres internaciona

In [6]:
sentiment = dspy.ChainOfThought("document -> sentiment")
response = sentiment(document=document)

print(response)

Prediction(
    reasoning='La inacción de los gobiernos en el cambio climático se debe a una combinación de factores, incluyendo intereses económicos a corto plazo, falta de regulaciones efectivas y desigualdad global. Los gobiernos deben asumir la responsabilidad y actuar con urgencia para abordar esta crisis.',
    sentiment='Negativa'
)


In [7]:
llama.inspect_history(n=2)





System message:

Your input fields are:
1. `document` (str)

Your output fields are:
1. `reasoning` (str)
2. `summary` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## document ## ]]
{document}

[[ ## reasoning ## ]]
{reasoning}

[[ ## summary ## ]]
{summary}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `document`, produce the fields `summary`.


User message:

[[ ## document ## ]]


Crítica hacia la Inacción de los Gobiernos en el Cambio Climático

  El cambio climático se ha convertido en una de las crisis más apremiantes de nuestro tiempo, pero los gobiernos parecen estar más preocupados por intereses económicos
  a corto plazo que por el bienestar del planeta. A pesar de las abrumadoras evidencias científicas, muchos líderes continúan posponiendo acciones decisivas.

  1. Promesas Vacías: A menudo, los gobiernos hacen promesas rimbombantes en cumbres internaciona

## **Agent in DSpy**

Importing the required libraries:

In [8]:
from pydantic import BaseModel
from typing import List, Callable

`Pydantic`es una librería que permite la validación de datos en Python. `BaseModel` es una clase de Pydantic que permite la creación de modelos de datos.

### *Signature in DSpy*

In [9]:
class Plan(dspy.Signature):
    """Produce a step by step plan to perform a task.
    The plan needs to be in markdown format and should be broken down into big steps (with ## headings) and sub-steps beneath those.
    When thinking about your plan, be sure to think about the tools at your disposal and include them in your plan.
    """

    task = dspy.InputField(prefix="Task", desc= "The task you have to create a plan for.")
    context = dspy.InputField(format=str, desc= "Any context that might be relevant to the task.")
    proposed_plan = dspy.OutputField(format=str, desc= "The step by step plan to perform the task.")

The `Plan` inherits from `dspy.Signature` which is the base class for defining input and output signatures for the agent.

### *Worker*

The `Worker` class is responsible for handling the plannig and execution of tasks assigned to the agent.

In [11]:
class Worker(dspy.Module):
    def __init__(self, role:str, tools:List):
        self.role = role
        self.tools = tools
        self.tool_descriptions = "\n".join([f"- {t.name}: {t.description}. To use this tool please provide: `{t.requires}`" for t in tools])
        self.plan = dspy.ChainOfThought(Plan)

    def forward(self, task:str):

        context = f"""
        Role: {self.role}
        Tools: {self.tool_descriptions}
        """

        input_args = dict(
            context=context,
            task=task
        )

        result = self.plan(**input_args)
        print(result.proposed_plan)
        

### *Tools*

Tools are specialized capabilities that can be used by the agent to perform specific tasks.

In [12]:
class Tool(BaseModel):
    name:str
    description:str
    requires:str
    function: Callable


Defininf the set of tools that the agent can use:

In [13]:
test_tools = [
    Tool(name="phone", description="a way of making phone calls", requires="phone_number", function=lambda x: "they've got time"),
    Tool(name="local business lookup", description="Look up businesses by category", requires="business category", function=lambda x: "Bills landscaping: 415-555-5555")
]

In [14]:
with dspy.context(lm=llama):
    Worker(role="assistant", tools=test_tools).forward("Buy a pizza")

### Step 1: Look Up Local Pizza Businesses
#### Use Local Business Lookup Tool
Use the local business lookup tool to find nearby pizza places.
* Input: `business category` = "pizza"
* Output: List of nearby pizza businesses

### Step 2: Make a Phone Call to Place an Order
#### Use Phone to Call Pizza Business
Call the phone number of the chosen pizza place to place an order.
* Input: `phone_number`
* Output: Confirmation of order details
